In [21]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import time
import pandas as pd

In [6]:
def val(P,x):
    return P[0]+P[1]*x+P[2]*x*x
def grad(P,x):
    return P[1]+P[2]*2*x

In [60]:
st=time.time()
N,A=50,0
breadth=1
ITER=800
T=0.01
fout=open('ret/simple_mean(N,A,breadth,ITER,T={},{},{},{},{}).txt'.format(N,A,breadth,ITER,T),'w')
#fout.write('N,A,breadth,ITER,T={},{},{},{},{}\n'.format(N,A,breadth,ITER,T))
fout.write('seed,opt,mean,median\n')
for seed in range(1,101):
    rnd.seed(seed)

    graph=[[j for j in range(max(0,i-breadth),min(i+breadth+1,N))] for i in range(N)]
    for a in range(A):
        graph.append([i for i in range(N)])
        for i in range(N):
            graph[i].append(len(graph)-1)
    Ps=[]
    for i in range(N):
        a=rnd.uniform(0,1)
        v=rnd.uniform(-100,100)
        #a(x-v)^2=ax^2-2avx+av^2
        Ps.append([a*v*v+rnd.uniform(-1000,1000),-2*a*v,a])

    locsversions=[]
    locs=[rnd.uniform(-400,400) for i in range(N)]
    for a in range(A):
        locs.append(10000)
    for rep in range(ITER):
        locsversions.append(locs.copy())
        nlocs=[locs[i]-T*grad(Ps[i],locs[i]) for i in range(N)]
        for a in range(A):
            nlocs.append(10000) #basic adversary: always send this big number to all neighbors
        for i in range(N):
            #TODO: use smarter use of neighbors' values?
            tmp=[nlocs[j] for j in graph[i]]
            tmp.sort()
            locs[i]=np.mean(tmp)#[1:-1])
    locsversions.append(locs)

    tP=[0 for i in range(3)]
    for P in Ps:
        for i in range(3):
            tP[i]+=P[i]
    for i in range(3):
        tP[i]/=N
    opt_x=-tP[1]/(2*tP[2])
    scrs=[val(tP,locsversions[ITER][i]) for i in range(N)]
    print('{},{},{},{}'.format(seed,val(tP,opt_x),np.mean(scrs),np.median(scrs)))
    fout.write('{},{},{},{}\n'.format(seed,val(tP,opt_x),np.mean(scrs),np.median(scrs)))
    print('time='+str(time.time()-st))
fout.close()

'''t1=np.linspace(min(locs[0:N])-20,max(locs[0:N])+20,50)
fig,ax=plt.subplots(1,3,figsize=(20,5))

ax[0].set_title('F_i, MEAN(F_i), agents\' final parameters')
ax[0].plot(t1,val(tP,t1),linewidth=3,alpha=0.75)
for P in Ps:
    ax[0].plot(t1,val(P,t1),linewidth=1,alpha=0.15)
ax[0].scatter(x=locs[0:N],y=[val(tP,locs[i]) for i in range(N)],c='black')

ax[1].set_title('agents\' parameters')
ax[1].set_xlabel('agent\'s position in graph')
for k in range(0,ITER):
    ax[1].plot(locsversions[k],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))
ax[1].plot([0,N],[opt_x,opt_x])

ax[2].set_title('agents\' scores')
ax[2].set_xlabel('agent\'s position in graph')
for k in range(0,ITER):
    ax[2].plot([val(tP,locsversions[k][i]) for i in range(N)],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))
ax[2].plot([0,N],[val(tP,opt_x),val(tP,opt_x)])

#plt.savefig('N,A,ITER,breadth={},{},{},{}'.format(N,A,ITER,breadth)+'.png')'''

1,1662.8954346209405,1726.4838192667514,1702.0851115286227
time=0.7465753555297852
2,2042.474950471248,2281.2060183626436,2291.3116873109702
time=1.5431208610534668
3,1478.4005600825353,1569.852328516511,1548.3454128804087
time=2.1747636795043945
4,1514.0228227724397,1539.33138740112,1525.8873363604566
time=2.7754151821136475
5,2004.9142775692517,2035.3150974829318,2016.6453822374315
time=3.408055543899536
6,1878.7940106579588,1888.665958579936,1882.5809565339878
time=4.0566840171813965
7,1361.824926502943,1379.3718505719662,1370.1917869913364
time=4.6833271980285645
8,1306.686798255798,1354.9445505767194,1325.9702892588657
time=5.282986164093018
9,2090.260944820599,2246.0361454679182,2213.727472558923
time=5.936611175537109
10,1635.6923804040312,1779.6399260446374,1781.341004968133
time=6.552275657653809
11,1592.7901154069762,1648.8444387542581,1615.8464746643324
time=7.18891978263855
12,1238.525196998235,1317.552867493233,1314.9286661263623
time=7.808571100234985
13,1424.116878232913

"t1=np.linspace(min(locs[0:N])-20,max(locs[0:N])+20,50)\nfig,ax=plt.subplots(1,3,figsize=(20,5))\n\nax[0].set_title('F_i, MEAN(F_i), agents' final parameters')\nax[0].plot(t1,val(tP,t1),linewidth=3,alpha=0.75)\nfor P in Ps:\n    ax[0].plot(t1,val(P,t1),linewidth=1,alpha=0.15)\nax[0].scatter(x=locs[0:N],y=[val(tP,locs[i]) for i in range(N)],c='black')\n\nax[1].set_title('agents' parameters')\nax[1].set_xlabel('agent's position in graph')\nfor k in range(0,ITER):\n    ax[1].plot(locsversions[k],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))\nax[1].plot([0,N],[opt_x,opt_x])\n\nax[2].set_title('agents' scores')\nax[2].set_xlabel('agent's position in graph')\nfor k in range(0,ITER):\n    ax[2].plot([val(tP,locsversions[k][i]) for i in range(N)],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))\nax[2].plot([0,N],[val(tP,opt_x),val(tP,opt_x)])\n\n#plt.savefig('N,A,ITER,breadth={},{},{},{}'.format(N,A,ITER,breadth)+'.png')"

In [69]:
tmp={}
N,A,breadth,ITER,T=50,0,1,800,0.01
DFs={}
F1='ret/simple_mean(N,A,breadth,ITER,T={},{},{},{},{}).txt'.format(N,A,breadth,ITER,T)
print('F1='+F1)
fin=open(F1,'r')
txt=[l for l in fin]
#print(txt)
print('N,A,breadth,ITER,T={},{},{},{},{}\n'.format(N,A,breadth,ITER,T))
names=txt[0][0:-1].split(',')
infos={}
for n in names:
    infos[n]=[]
for i in range(1,len(txt)):
    info=list(map(lambda x:float(x),txt[i][0:-1].split(',')))
    info[0]=int(info[0])
    for j in range(len(info)):
        infos[names[j]].append(info[j])
DFs[F1]=pd.DataFrame(infos)
print(DFs[F1])
print(np.array(DFs[F1]['mean']-DFs[F1]['opt']))

F1=ret/simple_mean(N,A,breadth,ITER,T=50,0,1,800,0.01).txt
N,A,breadth,ITER,T=50,0,1,800,0.01

    seed          opt         mean       median
0      1  1662.895435  1726.483819  1702.085112
1      2  2042.474950  2281.206018  2291.311687
2      3  1478.400560  1569.852329  1548.345413
3      4  1514.022823  1539.331387  1525.887336
4      5  2004.914278  2035.315097  2016.645382
..   ...          ...          ...          ...
95    96  1571.335046  1583.671778  1576.330434
96    97  1927.904582  1996.235245  1951.850953
97    98  2020.810714  2119.653622  2044.955432
98    99  1597.365148  1684.606121  1661.185847
99   100  1688.407331  1727.172770  1700.711769

[100 rows x 4 columns]
[ 63.58838465 238.73106789  91.45176843  25.30856463  30.40081991
   9.87194792  17.54692407  48.25775232 155.77520065 143.94754564
  56.05432335  79.02767049  86.20832684  55.17727695  54.80621198
  22.90864899  20.84723763  47.59953133  32.71047094  26.8139074
  79.81073902 331.42461465 109.20191015  1